#Analyzing the NYC Subway Dataset

##Background Information

In this project, we look at the NYC Subway data and figure out if more people ride the subway when it is raining versus when it is not raining.
The dataset that is available contains data about turnstiles at different stations as well as weather conditions.

##Statistical Test

###Independent, dependent variables

First, let's see what kind of data is in the dataset by exploring the column names.

In [1]:
import pandas as pd
filename='data/improved-dataset/turnstile_weather_v2.csv'
turnstile_weather = pd.read_csv(filename)
print list(turnstile_weather.columns.values)

['UNIT', 'DATEn', 'TIMEn', 'ENTRIESn', 'EXITSn', 'ENTRIESn_hourly', 'EXITSn_hourly', 'datetime', 'hour', 'day_week', 'weekday', 'station', 'latitude', 'longitude', 'conds', 'fog', 'precipi', 'pressurei', 'rain', 'tempi', 'wspdi', 'meanprecipi', 'meanpressurei', 'meantempi', 'meanwspdi', 'weather_lat', 'weather_lon']


The purpose of this project is to find out if more people use the subway when it is raining. Fortunately, there is a column rain that indicates if it was raining at a certain time and location. According to the documentation it is an indicator (0 or 1). 
How to measure if the subway is used more? Well, the data contains the number of entries at some units where a unit is a data collector of multiple turnstiles. ENTRIESn is the reading of cummulative turnstile entries from the remote unit and ENTRIESn_hourly indicates the difference in ENTRIES from the previous REGULAR reading. It seems like ENTRIESn_hourly is a good measure of how many people will use the subway.

Therefore, the independent variable is the rain. It can either be a 0 (no rain) or 1 (rain)
The dependent variable is the the number of entries since the last reading (ENTRIESn_hourly)

### Hypothesis and chosen test

The data that is provided is a sample. With the hypothesis tests we want to explore if we can conclude that the subway is used more when it is raining. 
Let μR and μN respectively be the population means for ENTRIESn_hourly when it is not raining and when it is raining. Let H0 and Ha respectively be the 0 and alternative hypothesis. The hypotheses can be written as:

H0 : μR = μN

Ha : μR > μN

When comparing means of populations, multiple tests can be used. A t-test or z-test can be used if the data is assumed to be normally distributed. 
However, the data from our sample is not normally distributed if we look at the plot below. Mann–Whitney´s U test is a nonparametrict test that can be used if we don't assume that the data follows a certain distribution. Because we don´ t assume any underlying distribution this test will be used. The test will be one tailed, because it is only interesting to know if **more** people use the subway when it is raining. The p critical value is 0.05, since this is quite common.

![Histogram of rain vs norain values](plots/histogram_rain_norain.png)

### Results from executing the test

Below the results from the test can be found.

In [2]:
import numpy as np
import scipy.stats
rain = turnstile_weather['ENTRIESn_hourly'][turnstile_weather['rain'] == 1]
norain = turnstile_weather['ENTRIESn_hourly'][turnstile_weather['rain'] == 0]
print "Norain mean + std:", np.mean(norain), np.std(norain)
print "Rain mean + std:", np.mean(rain), np.std(rain)
U, p = scipy.stats.mannwhitneyu(rain, norain)
print "U-statistic:", U
print "p-value:", p

Norain mean + std: 1845.53943866 2878.72731433
Rain mean + std: 2028.19603547 3189.26699226
U-statistic: 153635120.5
p-value: 2.74106957124e-06


The mannwhitney u test is per default one-tailed.  As can be seen, the p-value is a lot smaller than the p critical value of 0.05. Therefore it is very unlikely that the population means are equal, so the alternative hypothesis applies: more people ride the subway when it is raining.

##Linear Regression

Machine learning is a branch of artificial intelligence focused on constructing systems that learn from large amounts of data to make predictions.
Linear Regression is one of the most used models in machine learning.

### Regression model
To create a lineair regression model I will use the Ordinary Least Squares (OLS) from Statsmodels. 

### Features 

I have used the following features in the regression model

* rain: because in the statistical test, it is clear that rain matters
* hour: because there are probably more people taking the subway in peak hours
* meantempi: because there are probably less travelers when it is very cold or hot
* weekday: 
* UNIT (dummy variable): because when I used this, it drastically improved my R2 value.

### Parameters of non-dummy features and R-squared

below the parameters of the non-dummy features and R-squared are shown

In [4]:
import matplotlib.pyplot as plt
import statsmodels.api as sm

def linear_regression(features, values):
    """
    Perform linear regression given a data set with an arbitrary number of features. 
    This can be the same code as in the lesson #3 exercise.
    """ 
    features = sm.add_constant(features)
    model = sm.OLS(values, features)
    results = model.fit()
    intercept = results.params[0]
    params = results.params[1:]    
    return intercept, params
    

def compute_r_squared(data, predictions):
    '''
    Calculates R sqared based on given data and predictions
    '''
    SST = ((data - np.mean(data)) **2).sum()
    SSREG = ((predictions - data) **2).sum()
    r_squared = 1 - SSREG / SST
    
    return r_squared

filename='data/improved-dataset/turnstile_weather_v2.csv'
turnstile_weather = pd.read_csv(filename)
features = turnstile_weather[['rain', 'hour', 'meantempi', 'weekday']]
dummy_units = pd.get_dummies(turnstile_weather['UNIT'], prefix='unit')
features = features.join(dummy_units)
values = turnstile_weather['ENTRIESn_hourly']
intercept, params = linear_regression(features, values)
predictions = intercept + np.dot(features, params)
print "params", params
print "r-squared:", compute_r_squared(values, predictions)

 r-squared: 0.482340215273


2.6 What does this R2 value mean for the goodness of fit for your regression model? Do you think this linear model to predict ridership is appropriate for this dataset, given this R2  value?


##Visualization



##Conclusion

##Reflection